In [125]:
file_name_sat = 'S2_SR_USA_2017_2024_SCL.csv'
#file_name_sat = 'S2_TOArefl_USA_2017_2024.csv'
file_name_insitu = 'NEON_LAI_USA_2017_2024.csv'

In [126]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

import os

from sklearn import metrics

from variables import COORDS as coords

### 1. Datos de satelite

#### Cargamos el dataset de datos de satelite

Para entorno **local**

In [127]:
# Definimos ruta de los archivos.
datadir_path = os.path.join('..','data')
dataset_path = os.path.join(datadir_path, file_name_sat)

**Importamos el dataset** con Pandas y visualizamos las primeras instancias

In [128]:
df_sat = pd.read_csv(dataset_path, sep=',')

# Eliminamos columna innecesaria de indices importado
if 'Unnamed: 0' in df_sat.columns: # Comprobamos si 'Unnamed: 0' está en las columnas
    # Eliminar la columna 'Unnamed: 0'
    df_sat.drop(columns=['Unnamed: 0'], inplace=True) 
df_sat.head(5)

,id,longitude,latitude,time,B1,B2,B3,B4,B5,B6,...,B8,B8A,B9,B11,B12,SCL,QA10,QA20,QA60,Site_sat
0,20171222T182749_20171222T183128_T12TUK,-112.452455,40.177600,1513967574868,3597.0,3704.0,3788.0,3972.0,4316.0,4280.0,...,4272.0,4349.0,4724.0,1167.0,1091.0,11.0,NaN,NaN,0,Onaqui Ault
1,20171118T170559_20171118T170554_T15TYL,-89.586378,45.508922,1511025045890,8393.0,8072.0,7776.0,7568.0,7652.0,7527.0,...,7632.0,7314.0,10924.0,2087.0,2396.0,8.0,NaN,NaN,1024,Steigerwaldt Land Services
2,20171225T165709_20171225T165708_T15TYL,-89.586378,45.508922,1514221250709,3381.0,3469.0,3256.0,3317.0,3473.0,3863.0,...,4376.0,4062.0,3881.0,2315.0,1911.0,9.0,NaN,NaN,2048,Steigerwaldt Land Services
3,20171118T170559_20171118T170554_T15SUD,-95.192225,39.040513,1511025161923,7635.0,7848.0,7632.0,7480.0,7621.0,7438.0,...,7944.0,7295.0,10781.0,1901.0,2098.0,8.0,NaN,NaN,1024,University of Kansas Field Site
4,20171118T170559_20171118T170554_T15TYM,-89.537510,46.234042,1511025031342,5584.0,5352.0,5124.0,5016.0,5094.0,5038.0,...,4980.0,4760.0,6066.0,3001.0,2600.0,9.0,NaN,NaN,1024,Underc


Convertimos a tipo **datetime** la columna 'time' que está en formato Unix epoch time.

In [129]:
#df_sat['time'] = df_sat['time'].apply(lambda x: dt.datetime.utcfromtimestamp(x / 1000))
df_sat['time'] = pd.to_datetime(df_sat['time'], unit='ms')

**Merge de data sets**. Para cada site identificamos las medidas in_situ mas cercanas a la fecha de adquisicion del satélite

### 2. Datos de campo

Para entorno **local**

In [130]:
# Definimos ruta de los archivos.
#datadir_path = os.path.join('..','data')
dataset_path_insitu = os.path.join(datadir_path, file_name_insitu)

**Importamos el dataset** con Pandas y visualizamos las primeras instancias

In [131]:
df_insitu = pd.read_csv(dataset_path_insitu, sep=',')

# Eliminamos columna innecesaria de indices importado
if 'Unnamed: 0' in df_insitu.columns: # Comprobamos si 'Unnamed: 0' está en las columnas
    # Eliminar la columna 'Unnamed: 0'
    df_insitu.drop(columns=['Unnamed: 0'], inplace=True) 
df_insitu.head(5)

,GBOV_ID,Site,GROUND_DATA_PI,GROUND_DATA_PIs_Email,GBOV_Email,Network,Elevation,IGBP_class,Lat_IS,Lon_IS,...,LAI_Miller_down_err,LAI_Warren_down_err,LAIe_Miller_down_err,LAIe_Warren_down_err,clumping_Miller_down,clumping_Warren_down,clumping_Miller_up_err,clumping_Warren_up_err,clumping_Miller_down_err,clumping_Warren_down_err
0,GBOV_RM7_1223,Harvard Forest,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,351,Mixed Forest,42.5378,-72.171501,...,0.027,0.04,0.017,0.022,0.75,0.80,0.029,0.04,0.11,0.19
1,GBOV_RM7_1223,Harvard Forest,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,351,Mixed Forest,42.5378,-72.171501,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GBOV_RM7_1235,Harvard Forest,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,351,Mixed Forest,42.5378,-72.171501,...,0.11,0.18,0.07,0.11,0.76,0.74,0.033,0.05,0.08,0.17
3,GBOV_RM7_1235,Harvard Forest,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,351,Mixed Forest,42.5378,-72.171501,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GBOV_RM7_1244,Harvard Forest,Courtney Meier,cmeier@battelleecology.org,support-copernicus-gbov@acri-st.fr,NEON,351,Mixed Forest,42.5378,-72.171501,...,0.21,0.32,0.14,0.22,0.64,0.67,0.035,0.06,0.07,0.16


Quitamos el **dato UTC** a la columna fecha para estandarizarla con el dataset de satelite.

In [132]:
df_insitu['TIME_IS'] = pd.to_datetime(df_insitu['TIME_IS']).dt.tz_localize(None)
df_insitu[['Site','TIME_IS']].head(2)

,Site,TIME_IS
0,Harvard Forest,2018-08-13 00:00:00
1,Harvard Forest,2018-08-13 15:10:00


### 3. Combinación de los datasets

**Merge de data sets**. Para cada site identificamos las medidas in_situ mas cercanas a la fecha de adquisicion del satélite

In [133]:
df_list = []
days_diff_max = 5 # Diferencia maxima (en días) entre la medida de satelite y la medida in situ

# Itera sobre cada coordenada
for site, lat, lon in coords:
    # Filtra el DataFrame de df_insitu y df_sat según el 'Site'
    df_insitu_site = df_insitu[df_insitu['Site'] == site].copy()
    df_sat_site = df_sat[df_sat['Site_sat'] == site].copy()

    rows = []

    # Itera sobre los datos de satélite y sobre los datos in situ
    for index_sat, date_sat in df_sat_site['time'].items():
        for index_insitu, date_insitu in df_insitu_site['TIME_IS'].items():
            # Calcula la diferencia en días entre las fechas
            dif = (date_insitu - date_sat) / np.timedelta64(1, 'D')

            if abs(dif) <= days_diff_max: # Si la diferencia en dias es menor...
                # Crea una lista con la fila combinada de ambas fuentes de datos
                row = list(df_sat_site.loc[index_sat]) + list(df_insitu_site.loc[index_insitu])
                rows.append(row)

    # Si se encontraron filas, procesa los datos
    if rows:
        # Crear el DataFrame con las filas acumuladas
        df_cross_site = pd.DataFrame(rows, columns=list(df_sat_site.columns) + list(df_insitu_site.columns))

        # Añadir una columna 'delta' con la diferencia en tiempo entre el satélite y las mediciones in situ
        df_cross_site['delta'] = abs(df_cross_site['time'] - df_cross_site['TIME_IS'])

        # Añadir las fechas separadas de 'date_sat' y 'date_insitu'
        df_cross_site['date_sat'] = df_cross_site['time'].dt.date
        df_cross_site['date_insitu'] = df_cross_site['TIME_IS'].dt.date

        # Ordenar y eliminar duplicados por 'date_insitu'
        df_cross_site = df_cross_site.sort_values(by=['date_insitu', 'delta']).drop_duplicates(subset='date_insitu', keep='first')

        # Ordenar y eliminar duplicados por 'date_sat'
        df_cross_site = df_cross_site.sort_values(by=['date_sat', 'delta']).drop_duplicates(subset='date_sat', keep='first')


        # Agregar el DataFrame de este sitio a la lista
        df_list.append(df_cross_site)

# Combinar todos los DataFrames en uno solo
df_cross = pd.concat(df_list, axis=0, ignore_index=True)

Nos quedamos con los datos del 2017 al 2022. Reservamos los del 2023 para test final.

In [134]:
train_set = df_cross[pd.to_datetime(df_cross['date_insitu']) < pd.to_datetime('2023-01-01')]

In [135]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

**Eliminamos columnas innecesarias.**  
QA10 y QA20 son columnas vacias.  
También descartamos las bandas de 60m que corresponden a aerosoles y vapor de agua: B1 y B9

In [136]:
columns_to_exclude = ['B1', 'B9', 'QA10', 'QA20']

#pipe_exclude_columns = ColumnTransformer([("excluye","drop", columns_to_exclude)], remainder = "passthrough")

In [137]:
pipe_exclude_columns

ColumnTransformer(remainder='passthrough',
                  transformers=[('excluye', 'drop',
                                 ['B1', 'B9', 'QA10', 'QA20'])])

In [138]:
#pipe_train_compacto = pipe_exclude_columns.fit_transform(train_set)

In [139]:
#df_check = pd.DataFrame(pipe_train_compacto, columns = pipe_exclude_columns.get_feature_names_out())
#df_check

**Eliminamos las filas** (espectros) correspondientes a **pixeles con nubes**  
Los pixes con nubes corresponden a: QA60 == 512 or QA60 == 1024 or QA60 == 2048  
Por lo tanto nos quedamos solo con los pixeles libres de nubes: QA60 == 0

In [140]:
def remove_QA60_zero(X):
    return X[X['QA60'] == 0]

In [141]:
'''from sklearn.base import BaseEstimator, TransformerMixin

class RemoveQA60Zero(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # Nada que ajustar

    def transform(self, X):
        # Asegúrate de que "QA60" está en las columnas
        if "QA60" in X.columns:
            return X[X["QA60"] != 0].copy()
        else:
            raise ValueError("La columna 'QA60' no está en el DataFrame")'''

'from sklearn.base import BaseEstimator, TransformerMixin\n\nclass RemoveQA60Zero(BaseEstimator, TransformerMixin):\n    def fit(self, X, y=None):\n        return self  # Nada que ajustar\n\n    def transform(self, X):\n        # Asegúrate de que "QA60" está en las columnas\n        if "QA60" in X.columns:\n            return X[X["QA60"] != 0].copy()\n        else:\n            raise ValueError("La columna \'QA60\' no está en el DataFrame")'

In [142]:
pipeline = Pipeline([
    ('remove_QA60_zero', FunctionTransformer(remove_QA60_zero, feature_names_out='one-to-one')),
    ('pipe_exclude_columns', ColumnTransformer([("excluye","drop", columns_to_exclude)], remainder = "passthrough"))
])

In [143]:
resultado = pipeline.fit_transform(train_set)

In [144]:
df_check = pd.DataFrame(resultado, columns = pipeline.get_feature_names_out())
df_check

,remainder__id,remainder__longitude,remainder__latitude,remainder__time,remainder__B2,remainder__B3,remainder__B4,remainder__B5,remainder__B6,remainder__B7,...,remainder__LAIe_Warren_down_err,remainder__clumping_Miller_down,remainder__clumping_Warren_down,remainder__clumping_Miller_up_err,remainder__clumping_Warren_up_err,remainder__clumping_Miller_down_err,remainder__clumping_Warren_down_err,remainder__delta,remainder__date_sat,remainder__date_insitu
0,20190508T153819_20190508T154933_T18TYN,-72.171458,42.537834,2019-05-08 15:51:23.573,1700.0,1986.0,2260.0,2746.0,3233.0,3397.0,...,0.013,0.81,0.92,0.04,0.05,0.11,0.18,0 days 10:02:23.573000,2019-05-08,2019-05-08
1,20190801T153911_20190801T154113_T18TYN,-72.171458,42.537834,2019-08-01 15:51:19.875,176.0,422.0,188.0,757.0,3112.0,3827.0,...,0.024,0.78,0.78,0.03,0.05,0.08,0.15,2 days 09:45:19.875000,2019-08-01,2019-07-30
2,20190826T153819_20190826T154455_T18TYN,-72.171458,42.537834,2019-08-26 15:51:19.743,204.0,405.0,182.0,711.0,2746.0,3450.0,...,0.017,0.92,0.96,0.029,0.05,0.06,0.09,0 days 14:23:40.257000,2019-08-26,2019-08-27
3,20190925T153959_20190925T155023_T18TYN,-72.171458,42.537834,2019-09-25 15:51:15.187,175.0,373.0,204.0,792.0,2388.0,3075.0,...,0.04,0.81,0.9,0.028,0.04,0.12,0.19,1 days 08:10:15.187000,2019-09-25,2019-09-24
4,20191020T154301_20191020T154502_T18TYN,-72.171458,42.537834,2019-10-20 15:51:18.518,1044.0,1110.0,1056.0,1435.0,1858.0,2012.0,...,0.012,0.9,0.95,0.027,0.04,0.2,0.29,1 days 15:40:41.482000,2019-10-20,2019-10-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,20220612T155911_20220612T161136_T17RMM,-81.436323,28.125084,2022-06-12 16:15:39.069,9512.0,8820.0,8448.0,9226.0,8411.0,8016.0,...,0.0760044207719843,0.921248,0.91032,-999.0,-999.0,0.043896,0.09073,0 days 19:33:20.931000,2022-06-12,2022-06-13
1063,20220627T155829_20220627T161037_T17RMM,-81.436323,28.125084,2022-06-27 16:15:34.147,8452.0,7834.0,7494.0,8316.0,7890.0,7671.0,...,0.0746725061829818,0.904628,0.904155,-999.0,-999.0,0.042031,0.071971,0 days 21:38:25.853000,2022-06-27,2022-06-28
1064,20220712T155921_20220712T161132_T17RMM,-81.436323,28.125084,2022-07-12 16:15:41.829,635.0,1009.0,870.0,1681.0,2862.0,3219.0,...,0.0787403291543463,0.891503,0.896217,-999.0,-999.0,0.045395,0.080408,0 days 18:33:18.171000,2022-07-12,2022-07-13
1065,20220811T155921_20220811T160715_T17RMM,-81.436323,28.125084,2022-08-11 16:15:41.161,2649.0,2586.0,2307.0,2795.0,3434.0,3570.0,...,0.0502821598173343,0.903323,0.923436,-999.0,-999.0,0.058342,0.108301,1 days 05:13:41.161000,2022-08-11,2022-08-10


In [145]:
pipeline

C:\Users\josea\AppData\Roaming\Python\Python312\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('remove_QA60_zero',
                 FunctionTransformer(feature_names_out='one-to-one',
                                     func=<function remove_QA60_zero at 0x00000253F51AE0C0>)),
                ('pipe_exclude_columns',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('excluye', 'drop',
                                                  ['B1', 'B9', 'QA10',
                                                   'QA20'])]))])